<신경망 구성의 단계적 절차>

1. 매개변수 설정(입력노드, 은닉층 노드, 출력층 노드, 학습률, 학습횟수(epoch))


2. 학습용 train과 평가용 test 데이터 셋을 추출


3. 다중 글자를 분류하여 학습하고 테스트 하는 것이기 때문에 다중 클래스 분류를 위해서 원-핫인코딩 방식을 채택


4. 순전파 은닉층에서 사용 할 활성화 함수 정의(시그모이드 함수, 하이퍼볼릭 탄젠트, Relu)


5. 역전파 은닉층에서 사용 할 활성화 함수 미분 정의


6. 출력층에서 사용 할 활성화 함수(소프트 맥스=> 다중 클래스 분류에서 사용하기 용이함)


7. 학습의 정확도와 평가의 정확도를 알기 위해서 정확도 함수를 정의


8. 신경망을 구성하여 순전파로 오차를 구하고, 이 오차의 미분을 통하여 기울기를 구함. 이를 통해 가중치와 편향을 업데이트 하여 학습 함.

신경망을 구성하면 서 필요한 라이브러리들 import
csv파일을 읽어 들일 수 있는 csv
현재 디렉토리에 있는 파일에서 글자데이터를 찾고 train과 test에서 필요한 파일들을 찾을 수 있는 os
신경망의 행렬 계산을 쉽게 해줄 수 있는 numpy

In [17]:
import csv
import os
import numpy as np

1.1 신경망에서 필수적으로 필요한 각각의 매개변수들을 정의함.

1.2 csv파일로 만든 train 데이터 셋과 test 데이터 셋을 메모리에 로드 할 수 있게 하는 함수를 정의 함.(입력값: 픽셀 데이터, 출력 값: 레이블)로 구성됨.

1.3 CSV 파일의 첫 번째 열은 클래스 레이블로 저장되어 있고 
나머지 열은 각 글자를 표현하는 픽셀 데이터로 구성되어 있어 다중 클래스 분류에서 사용하기 용이하게 함.
예를 들어 "가"라는 글자는 [1, 0, 0, ..., 0] 형태로 변환 이를 통하여 소프트맥스 활성화 함수를 사용하고 예측 확률이 가장 높은 클래스를 선택하기 위함.

In [ ]:
# 신경망 하이퍼파라미터 설정
input_size = 4096           # 입력 크기: 64x64 이미지의 총 픽셀 수 (4096)
hidden_layers = [256,240,220]  # 은닉층 노드 수
output_size = 111            # 출력층 노드 수: 분류할 클래스 수
learning_rate = 0.0001       # 학습률: 가중치 업데이트의 크기
epochs = 300                  # 학습 데이터 전체를 얼마나 반복할지

#데이터 로드 함수 
def load_data(sub_dir, file_name, output_size):
    current_dir = os.getcwd()  # 현재 작업 디렉토리 경로 가져오기
    file_path = os.path.join(current_dir, '한글글자데이터', sub_dir, file_name)   # 데이터 파일 경로

    pixel_data = [] # 픽셀 데이터
    labels = [] # 레이블(정답) 데이터

    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            # 문자열을 정수로 변환
            label = int(row[0].strip()) - 1  # 첫 번째 열은 레이블 (0~110)
            row_data = [float(value) / 255.0 for value in row[1:]]  # 픽셀 데이터 정규화 (0~255 -> 0~1)
            pixel_data.append(row_data)  # 픽셀 데이터를 리스트에 추가

            # 원핫 인코딩 생성
            one_hot_label = [0] * output_size  # 출력 크기만큼 0으로 채운 리스트 생성
            one_hot_label[label] = 1  # 해당 레이블 인덱스를 1로 설정
            labels.append(one_hot_label)  # 원핫 인코딩된 레이블을 리스트에 추가

    # 데이터를 numpy 배열로 변환하여 반환
    return np.array(pixel_data), np.array(labels)


강의 시간 때 배웠던 활성화 함수들을 정의하였다.
1. 시그모이드 함수
2. 하이퍼볼릭 탄젠트 함수
3. Relu 함수
4. 다중 클래스 분류를 위한 소프트 맥스 함수(출력층에서만 사용)

In [19]:
"활성화함수 정의"

#시그모이드 함수
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  # 시그모이드 함수: 1 / (1 + e^(-x))

# Relu(순전파 사용)
def relu(x):
    return np.maximum(0, x)  # ReLU 함수: 입력이 음수이면 0, 양수이면 그대로 출력

# tanh(순전파 사용)
def tanh(x):
    return np.tanh(x)  # tanh 함수: (e^x - e^(-x)) / (e^x + e^(-x))

#시그모이드 미분(역전파 사용)
def sigmoid_derivative(x):  
    return sigmoid(x) * (1 - sigmoid(x))  # 시그모이드 미분: sigmoid(x) * (1 - sigmoid(x))

# Relu 미분(역전파 사용)
def relu_derivative(x):
    return np.where(x > 0, 1, 0)  # ReLU 미분: 입력이 양수이면 1, 음수이면 0

# tanh 미분(역전파 사용)
def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2  # tanh 미분: 1 - tanh(x)^2

# 출력층 활성화함수 정의
def softmax(x):
    exp_x = np.exp(x)  # 입력값 x에 대한 지수 함수 계산
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)  # softmax: 확률로 변환

신경망 모델의 순전파를 통하여 오차를 구하기 위해서 평균제곱 오차를 사용하였다.
평균 제곱 오차의 계산은 (예측값-실제값)^2와 같이 계산이 되고, 이를 통하여 오차를 계산하고 역전파에서 가중치와 편향을 업데이트한다.


In [20]:
"손실함수 정의"

# 평균제곱오차
def MSE(y_true, y_pred):    # 실제값과 예측값을 사용하여 평균제곱오차 계산
    return np.mean((y_true-y_pred) ** 2)

모델의 정확도를 통하여 학습의 정확도와 평가용 테스트 셋을 준비한(test데이터)의 정확도를 통하여 모델이 얼마나 학습을 잘하고 있는지 정확도 함수를 정의하였다.

In [21]:
"정확도 계산"

# 정확도 계산 함수 정의
def accuracy(y_true, y_pred):   # 정답과 예측값을 받아서 정확도 계산
    if y_true.ndim > 1:      
        y_true = np.argmax(y_true, axis=1)  #원한 인코딩 정수 변환
    if y_pred.ndim > 1:  
        y_pred = np.argmax(y_pred, axis=1)  # 원핫 인코딩 정수 변환
    correct = np.sum(y_true == y_pred)  # 정답과 예측이 일치하는 개수 계산
    return correct / len(y_true)  # 정확도 계산

신경망 구성

1. 순전파
입력층(4096)의 데이터를 순전파하여 출력층(111)의 예측값을 계산하고 이를 실제값과 비교하여 손실 함수로 오차를 계산함

2. 역전파
순전파에서 계산된 오차를 토대로 손실 함수의 기울기를 계산하고 가중치와 편향을 조정 함.

3. 학습
경사하강법을 통한 가중치와 편향의 업데이트가 여러번의 에폭(학습과정)을 통하여 모델을 조정해 나감으로써 모델의 예측 성능을 점진적으로 높임.
(학습 과정에서 경사하강법으로 인한 기울기 소실 문제가 발생하는 것을 대비하여 가중치의 he초기화를 통하여 기울기 소실 방지)

4. 평가
평가용 데이터 셋으로 만들어진 학습 데이터에서는 사용되지 않은 폰트들과 노이즈가 섞인 데이터를 학습 데이터와 비교하며 모델이 범용성이 있는지를 보임.

In [22]:
# 신경망 클래스 정의
class NeuralNetwork:
    def __init__(self, input_size, hidden_layers, output_size, learning_rate):
        self.learning_rate = learning_rate  
        self.weights = [] 
        self.biases = []  
        
        
        layer_sizes = [input_size] + hidden_layers + [output_size]  # 입력, 은닉, 출력층 노드 수
        
        # 각 레이어 가중치와 편향 초기화
        for i in range(len(layer_sizes) - 1):
            weight = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2.0 / layer_sizes[i])  # 심층 신경망에서 기울기 소실 방지를 위한 he 초기화
            bias = np.zeros((1, layer_sizes[i + 1]))  # 편향 초기화
            self.weights.append(weight)
            self.biases.append(bias)

    # 순전파
    def forward(self, x, training=True):
        self.activations = [x]  # 입력 데이터를 첫 번째 활성화 값으로 설정
        # 은닉층 활성화 함수(Relu) 계산
        for i in range(len(self.weights) - 1):  # 은닉층 수만큼 반복
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]  # 
            a = relu(z)  # 활성화 함수 적용
            self.activations.append(a)
        # 출력층 활성화 함수(소프트 맥스)계산
        z = np.dot(self.activations[-1], self.weights[-1]) + self.biases[-1]
        a = softmax(z)  # 소프트맥스 활성화 함수 적용
        self.activations.append(a)
        return self.activations[-1] # 출력층 활성화 값 반환

    # 역전파
    def backward(self, y_true):
        # 출력층 오차
        deltas = [self.activations[-1] - y_true]
        
        # 은닉층에서의 오차 전파(출력층-> 은닉층 , 은닉층->입력층)
        for i in reversed(range(len(self.weights) - 1)):
            delta = deltas[-1].dot(self.weights[i + 1].T) * relu_derivative(self.activations[i + 1])
            deltas.append(delta)
        deltas.reverse()  # 순서를 원래대로 변경
        
        # 가중치 및 편향 업데이트
        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * self.activations[i].T.dot(deltas[i])
            self.biases[i] -= self.learning_rate * np.sum(deltas[i], axis=0, keepdims=True)


    # 학습
    def train(self, x, y, epochs, learning_rate):
        self.learning_rate = learning_rate

        # 클래스별 데이터 인덱스 생성
        class_indices = {label: np.where(np.argmax(y, axis=1) == label)[0] for label in range(y.shape[1])}

        for epoch in range(epochs): 
            if (epoch + 1) % 10 == 0:   #주기적 모델 평가
                print("모델 범용성 평가")
                self.evaluate(x, y) 

            total_loss = 0  # 총 손실 초기화
            correct_predictions = 0  # 정확한 예측 수 초기화

            # 클래스별 학습 루프
            for class_label, indices in class_indices.items():
                np.random.shuffle(indices)  # 클래스 데이터를 섞어서 순차적으로 학습
                class_x = x[indices]
                class_y = y[indices]

                # 순전파
                output = self.forward(class_x, training=True)   
                loss = MSE(class_y, output) #실제값과 예측값의 손실 계산
                total_loss += loss  #총 손실 누적

                # 정확도 계산
                predictions = np.argmax(output, axis=1) #예측값
                true_labels = np.argmax(class_y, axis=1)    #실제값
                correct_predictions += np.sum(predictions == true_labels)   #정확한 예측 수 계산

                # 역전파
                self.backward(class_y)  

            # 평균 손실 및 학습 정확도 계산
            average_loss = total_loss / len(x)  # 전체 데이터 크기로 나눔
            train_accuracy = correct_predictions / len(x) * 100

            print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {average_loss}, Train Accuracy: {train_accuracy:.2f}%")

    # 평가 함수
    def evaluate(self, x, y):
        test_output = self.forward(x, training=False)   #테스트 데이터에 대한 예측값
        test_loss = MSE(y, test_output) #테스트 데이터에 대한 손실 계산
        test_predictions = np.argmax(test_output, axis=1)   #테스트 데이터 예측값
        test_true_labels = np.argmax(y, axis=1) #테스트 데이터 실제값
        test_accuracy = accuracy(test_true_labels, test_predictions)    #테스트 데이터 정확도 계산

        print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy * 100:.2f}%")
        return test_loss, test_accuracy

학습용 데이터셋 csv와 평가용 데이터셋 csv를 불러들여 픽셀 데이터와 레이블로 나눔

In [23]:
# 데이터 로드
train_data, train_labels = load_data('train', 'train_data.csv', output_size)
test_data, test_labels = load_data('test', 'test_data.csv', output_size)

신경망 객체를 생성하여 학습을 실행하는 코드

In [24]:
# 신경망 객체 생성
nn = NeuralNetwork(input_size, hidden_layers, output_size, learning_rate)
# 학습 실행
nn.train(train_data, train_labels, epochs, learning_rate)

Epoch 1/500, Train Loss: 0.0009033192926336561, Train Accuracy: 0.45%
Epoch 2/500, Train Loss: 0.000894929209100165, Train Accuracy: 0.18%
Epoch 3/500, Train Loss: 0.0008939113853505165, Train Accuracy: 0.18%
Epoch 4/500, Train Loss: 0.000893245263417225, Train Accuracy: 0.09%
Epoch 5/500, Train Loss: 0.0008927593905520471, Train Accuracy: 0.72%
Epoch 6/500, Train Loss: 0.0008924858069889452, Train Accuracy: 0.63%
Epoch 7/500, Train Loss: 0.0008920038374976716, Train Accuracy: 0.99%
Epoch 8/500, Train Loss: 0.0008915021045096777, Train Accuracy: 2.16%
Epoch 9/500, Train Loss: 0.0008910850539947611, Train Accuracy: 2.34%
모델 범용성 평가
Test Loss: 0.008894139690840845, Test Accuracy: 5.41%
Epoch 10/500, Train Loss: 0.0008906766016376303, Train Accuracy: 3.06%
Epoch 11/500, Train Loss: 0.0008899718273224545, Train Accuracy: 3.69%
Epoch 12/500, Train Loss: 0.0008893554395191868, Train Accuracy: 4.59%
Epoch 13/500, Train Loss: 0.000888552806734136, Train Accuracy: 4.86%
Epoch 14/500, Train Loss: